This program is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License, version 2, as published by the Free Software Foundation.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

Author: Alexander Young

### This script uses reconstructed geometries exported from GPlates to model time dependent submarine sediment thickness.

##### Input:
reconstructed coastlines e.g. reconstructed_0.00Ma.xy

reconstructed subduction zone topologies e.g. topology_subduction_boundaries_0.00Ma.xy

global sediment thickness grid GlobSed-v2_resample1deg-x-1deg.nc

##### Output:
coastline sediment thickness grid e.g. cs_stencilGrid_0Ma_500km.nc

abyssal plain sediment thickness grid e.g. sz_stencilGrid500km0Ma_.nc

present day sediment thickness distribution plot e.g. modeled_sediment_thickness_500km.pdf

total sediment thickness grid e.g. Modeled_TotalThickness_SedGrid_0Ma_500km.nc

time-dependent mean sediment thickness file e.g. SedModelFinal_0.0-580.0Ma_500km_Clipped5km.csv

##### Citations:
Dutkiewicz, A., Müller, R., Wang, X., O'callaghan, S., Cannon, J., and Wright, N., 2017, Predicting sediment thickness on vanished ocean crust since 200 Ma: Geochemistry, Geophysics, Geosystems, v. 18, no. 12, p. 4586-4603.

Straume, E., Gaina, C., Medvedev, S., Hochmuth, K., Gohl, K., Whittaker, J. M., Abdul Fattah, R., Doornenbal, J., and Hopper, J. R., 2019, GlobSed: Updated total sediment thickness in the world's oceans: Geochemistry, Geophysics, Geosystems, v. 20, no. 4, p. 1756-1772.

In [ ]:
import sys
import os
import os.path
import subprocess
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import gridspec
import scipy.stats as ss
import glob

# allow plots to appear within the notebook
% matplotlib inline

In [ ]:
# -- Extract present day sediment thickness along non-active margin coastlines from global sediment thickness grid
# set working directory and make if it does not exist
workDir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/SedimentModeling"
if not os.path.exists(workDir):
    os.makedirs(workDir)

# set stencil width - distance from slab in km
stencilWidth=500

age=0

# -- coastline and subduction zone input topologies
csFile="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/Coastlines/reconstructed_%d.00Ma.xy" %(age)
szFile="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/TopologiesExport/topology_subduction_boundaries_%d.00Ma.xy" %(age)

# -- sediment thickness input grid 
sedGrid="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/DependentInputs/GlobSed-v2_resample1deg-x-1deg.nc"

# -- sets output file names
cs_stencilGrid=workDir+"/cs_stencilGrid_%dkm.nc" %(stencilWidth)
sz_stencilGrid=workDir+"/sz_stencilGrid%dkm.nc" %(stencilWidth)
cs_SedGrid=workDir+"/cs_SedGrid_%dkm.nc" %(stencilWidth)
cs_SedGrid_xyz=workDir+"/cs_SedGrid_%dkm_mask_%d.xyz" %(stencilWidth, age)
    
# -- create cs stencil mask from coastlines input polygons
cmd="grdmask %s -Rg -I5d -m -NNaN/1/1 -S%dfk -Gtmp.nc" %(csFile, stencilWidth)
print cmd
os.system(cmd)

cmd="grdsample tmp.nc -Rg -I1d -G%s" %(cs_stencilGrid)
print cmd
os.system(cmd)

# -- create sz stencil mask from subduction zone input topologies
cmd="grdmask %s -Rg -I5d -m -N1/NaN/NaN -S%dfk -Gtmp2.nc" %(szFile, stencilWidth)
print cmd
os.system(cmd)

cmd="grdsample tmp2.nc -Rg -I1d -G%s" %(sz_stencilGrid)
print cmd
os.system(cmd)

# -- use cs stencil to extract topography values from grid
cmd="grdmath %s %s MUL = %s" %(sedGrid, cs_stencilGrid, cs_SedGrid)
print cmd
os.system(cmd)
    
# -- mask sz sed thickness using sz stencil 
cmd="grdmath %s %s MUL = %s" %(cs_SedGrid, sz_stencilGrid, cs_SedGrid)
print cmd
os.system(cmd)
    
# -- convert grid to xyz file
cmd="grd2xyz %s > %s" %(cs_SedGrid, cs_SedGrid_xyz)
print cmd
os.system(cmd)

# -- remove tmp files
cmd="rm tmp.nc tmp2.nc"
print cmd
os.system(cmd)

In [ ]:
# -- Extract sediment thickness from everywhere that is not within 'x' stencil width from coastlines
# -- sets output file names
non_cs_stencilGrid=workDir+"/non_cs_stencilGrid_%dkm.nc" %(stencilWidth)
non_cs_SedGrid=workDir+"/non_cs_SedGrid_%dkm.nc" %(stencilWidth)
non_cs_SedGrid_xyz=workDir+"/non_cs_SedGrid_%dkm_mask_%d.xyz" %(stencilWidth, age)

# -- invert cs_SedGrid 
cmd="grdclip %s -G%s -Sa0/1 -V" %(cs_SedGrid, non_cs_stencilGrid)
print cmd
os.system(cmd)

cmd="grdmath %s ISNAN = %s" %(non_cs_stencilGrid, non_cs_stencilGrid)
print cmd
os.system(cmd)

cmd="grdmath %s 0 NAN = %s" %(non_cs_stencilGrid, non_cs_stencilGrid)
print cmd
os.system(cmd)

# -- use cs mask to extract thickness values from grid
cmd="grdmath %s %s MUL = %s" %(sedGrid, non_cs_stencilGrid, non_cs_SedGrid)
print cmd
os.system(cmd)

# -- convert grid to xyz file
cmd="grd2xyz %s > %s" %(non_cs_SedGrid, non_cs_SedGrid_xyz)
print cmd
os.system(cmd)

In [ ]:
# load sediment thickness .xyz files we just created as dataframes 
cs_sed_in = pd.read_csv(cs_SedGrid_xyz, delimiter = "\t", header=None, names=['Long', 'Lat', 'Thickness_km'])
non_cs_sed_in = pd.read_csv(non_cs_SedGrid_xyz, delimiter = "\t", header=None, names=['Long', 'Lat', 'Thickness_km'])

# -- Drop all rows that have any NaN values
cs_sed_df = cs_sed_in[np.isfinite(cs_sed_in['Thickness_km'])]
non_cs_sed_df = non_cs_sed_in[np.isfinite(non_cs_sed_in['Thickness_km'])]

# -- fit exponential distribution to data
cs_loc, cs_scale=ss.expon.fit(cs_sed_df['Thickness_km'])
non_cs_loc, non_cs_scale=ss.expon.fit(non_cs_sed_df['Thickness_km'])

# -- Model sed thickness 
cs_sed_df['exponNumCol'] = np.random.exponential(scale=cs_scale, size=cs_sed_df.shape[0])
non_cs_sed_df['exponNumCol'] = np.random.exponential(scale=non_cs_scale, size=non_cs_sed_df.shape[0])

In [ ]:
# Plot observed and model sediment thickness
fig = plt.figure(figsize=(8,2.5))

font = {'family': 'helvetica',
        'color':  'black',
        'size': 10,
        'style': 'italic'
        }

font2 = {'family': 'helvetica',
        'color':  'black',
        'weight': 'bold',
        'size': 12,
        }

rX = np.linspace(0,18, 10000)
rP = ss.expon.pdf(rX, *(ss.expon.fit(cs_sed_df['Thickness_km'])))

#need to plot the normalized histogram with `normed=True`
ax1 = plt.subplot(121)
ax1.hist(cs_sed_df['Thickness_km'], normed=True, bins=50,
         linewidth=0.5, edgecolor='black', color='#5D158A')
ax1.plot(rX, rP, color='black', linewidth=3)
ax1.plot(rX, rP, color='#C650FB', linewidth=2)
plt.ylabel ('Density')

plt.xlim(0,18)
plt.ylim(0,1.8)

# expon_lambda= (cs_param[1:])
expon_lambda= np.round(cs_scale,1)

plt.text(0.1, 0.3,r"y = (x - %.2f) / %.2f" %(cs_loc, cs_scale ),
     horizontalalignment='left',
     verticalalignment='bottom',
     transform = ax1.transAxes, fontdict=font)

# plt.text(0.9, 0.85,"C.",
#      horizontalalignment='left',
#      verticalalignment='bottom',
#      transform = ax1.transAxes, fontdict=font2)

plt.xlabel ('Passive margin sediment thickness (km)')

rP = ss.expon.pdf(rX, *(ss.expon.fit(non_cs_sed_df['Thickness_km'])))
ax2 = plt.subplot(122)
ax2.hist(non_cs_sed_df['Thickness_km'], normed=True, bins=50,
         linewidth=0.5, edgecolor='black', color='#27B209')
ax2.plot(rX, rP, color='black', linewidth=3)
ax2.plot(rX, rP, color='#2FF903', linewidth=2)
ax2.tick_params(axis="x", bottom=True, top=False, labelbottom=True, labeltop=False)
ax2.yaxis.tick_right()
plt.setp(ax2.get_yticklabels(),visible=False)

plt.xlim(0,18)
plt.ylim(0,1.8)

# expon_lambda= (topo_param[1:])
expon_lambda= np.round(non_cs_scale,1)

plt.text(0.1, 0.3,r"y = (x - %.2f) / %.2f" %(non_cs_loc, non_cs_scale ),
     horizontalalignment='left',
     verticalalignment='bottom',
     transform = ax2.transAxes, fontdict=font)

# plt.text(0.9, 0.85,"D.",
#      horizontalalignment='left',
#      verticalalignment='bottom',
#      transform = ax2.transAxes, fontdict=font2)

plt.xlabel ('Non passive margin sediment thickness [km]')

plt.subplots_adjust(wspace = .045)

plt.savefig(workDir+'/modeled_sediment_thickness_%skm.pdf' %stencilWidth, format='pdf', bbox_inches = 'tight')


In [ ]:
# Now to model sediment thickness through time
# First, set path to output directory
outDir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/SedimentModeling"

# Set path to input files
csDir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/Coastlines"
topologyDir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/GPlatesFiles/TopologiesExport/"
COBGridDir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/DependentInputs/COB_masks"    

# Set path to output stencil directories - make if they dont exist
cs_Dir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/SedimentModeling/cs_Files"
if not os.path.exists(cs_Dir):
    os.makedirs(cs_Dir)

non_cs_Dir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/SedimentModeling/non_cs_Files"
if not os.path.exists(non_cs_Dir):
    os.makedirs(non_cs_Dir)
    
total_Dir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/SedimentModeling/TotalThickness"
if not os.path.exists(total_Dir):
    os.makedirs(total_Dir)


In [ ]:
# now, we need to create time dependent cs and abyssal plain stencils
# Iterate over time steps
reconstruction_times = np.arange(0, 580, 20)
for age in reconstruction_times:
    
    print ("Age is: ",round(age, 1), "Ma")

    # -- path for input
    csFile=csDir+"/reconstructed_%d.00Ma.xy" %(age)
    COBGrid=COBGridDir+"/COB_mask_global_%s.nc" %(age)
    szFile=topologyDir+"/topology_subduction_boundaries_%d.00Ma.xy" %(age)

    # -- sets output file names
    cs_stencilGrid=cs_Dir+"/cs_stencilGrid_%dMa_%dkm.nc" %(age, stencilWidth)
    cs_SedGrid_xyz=cs_Dir+"/cs_SedGrid_%dMa_mask_%dkm.xyz" %(age,stencilWidth)
    non_cs_stencilGrid=non_cs_Dir+"/non_cs_stencilGrid_%dMa_%dkm.nc" %(age, stencilWidth)
    non_cs_SedGrid_xyz=non_cs_Dir+"/non_cs_SedGrid_%dMa_mask_%dkm.xyz" %(age,stencilWidth)
    sz_stencilGrid=non_cs_Dir+"/sz_stencilGrid%dkm%dMa_.nc" %(stencilWidth,age)

    # -- create cs stencil mask from COB input polygons
    cmd="grdmask %s -Rg -I5d -m -NNaN/1/1 -S%dfk -Gtmp.nc" %(csFile, stencilWidth)
    print cmd
    os.system(cmd)

    cmd="grdsample tmp.nc -Rg -I1d -G%s" %(cs_stencilGrid)
    print cmd
    os.system(cmd)
   
    # -- create sz stencil mask from subduction zone input topologies
    cmd="grdmask %s -Rg -I5d -m -N1/NaN/NaN -S%dfk -Gtmp2.nc" %(szFile, stencilWidth)
    print cmd
    os.system(cmd)

    cmd="grdsample tmp2.nc -Rg -I1d -G%s" %(sz_stencilGrid)
    print cmd
    os.system(cmd)
        
    # -- mask cs stencil using sz stencil 
    cmd="grdmath %s %s MUL = %s" %(cs_stencilGrid, sz_stencilGrid, cs_stencilGrid)
    print cmd
    os.system(cmd)
    
    # -- mask cs stencil using cob mask
    cmd="grdmath -V %s %s OR = %s" %(cs_stencilGrid, COBGrid, cs_stencilGrid)
    print cmd
    os.system(cmd)
        
    # -- convert grid to xyz file
    cmd="grd2xyz %s > %s" %(cs_stencilGrid, cs_SedGrid_xyz)
    print cmd
    os.system(cmd)
    
    # -- invert cs stencil grid 
    cmd="grdclip %s -G%s -Sa0/1 -V" %(cs_stencilGrid, non_cs_stencilGrid)
    print cmd
    os.system(cmd)

    cmd="grdmath %s ISNAN = %s" %(non_cs_stencilGrid, non_cs_stencilGrid)
    print cmd
    os.system(cmd)

    cmd="grdmath %s 0 NAN = %s" %(non_cs_stencilGrid, non_cs_stencilGrid)
    print cmd
    os.system(cmd)
    
    # -- mask cs stencil using cob mask
    cmd="grdmath -V %s %s OR = %s" %(non_cs_stencilGrid, COBGrid, non_cs_SedGrid)
    print cmd
    os.system(cmd)

    # -- convert grid to xyz file
    cmd="grd2xyz %s > %s" %(non_cs_SedGrid, non_cs_SedGrid_xyz)
    print cmd
    os.system(cmd)
    
    # -- remove tmp files
    cmd="rm tmp.nc tmp2.nc"
    print cmd
    os.system(cmd)
    
    

In [ ]:
# -- Now populate the stencils with sediment thicknesses from our probability frequency distributions
# make empty lists to record modeled thickness information
TotalThickness_list=[]
mean_list=[]

# Clipping on; Dutkiewicz et al. (2017) showed that the present-day sed thickness may be anomalously large
# 1 yes, 0 no
Clip = 1.
clip_lim=5.

# set time interval and timestep
mintime=0.
maxtime=580.
timesetp=20
reconstruction_times = np.arange(mintime, maxtime, timesetp)
for age in reconstruction_times:
    print ("Age is: ",round(age, 1), "Ma")

    # -- infiles 
    cs_SedGrid_xyz=cs_Dir+"/cs_SedGrid_%dMa_mask_%dkm.xyz" %(age,stencilWidth)
    non_cs_SedGrid_xyz=non_cs_Dir+"/non_cs_SedGrid_%dMa_mask_%dkm.xyz" %(age,stencilWidth)

    # -- path for output
    Model_non_cs_SedGrid_xyz=non_cs_Dir+"/Modeled_non_cs_SedGrid_%dMa_%dkm.xyz" %(age,stencilWidth)
    Model_TotalThickness_SedGrid_xyz=total_Dir+"/Modeled_TotalThickness_SedGrid_%dMa_%dkm.xyz" %(age,stencilWidth)
    Model_TotalThickness_SedGrid=total_Dir+"/Modeled_TotalThickness_SedGrid_%dMa_%dkm.nc" %(age,stencilWidth)
    
    # -- Load in stencil grids
    cs_SedGrid_df = pd.read_csv(cs_SedGrid_xyz, delimiter = "\t", 
                                     header=None, names=['Long', 'Lat', 'Thickness_km'])

    non_cs_SedGrid_df = pd.read_csv(non_cs_SedGrid_xyz, delimiter = "\t", 
                                     header=None, names=['Long', 'Lat', 'Thickness_km'])
    
    # Model sed thickness
    model_cs_SedGrid = cs_SedGrid_df[np.isfinite(cs_SedGrid_df['Thickness_km'])]
    model_cs_SedGrid['exponNumCol'] = np.random.exponential(scale=cs_scale, size=model_cs_SedGrid.shape[0])
    model_cs_SedGrid.drop(['Thickness_km'], axis=1, inplace=True)
    if Clip == 1:
        Model_cs_SedGrid_xyz=cs_Dir+"/Modeled_cs_SedGrid_%dMa_%dkm_Clipped%dkm.xyz" %(age, stencilWidth, clip_lim)
        model_cs_SedGrid['exponNumCol'] = np.where(model_cs_SedGrid['exponNumCol'] >= clip_lim, clip_lim, model_cs_SedGrid['exponNumCol'])
        model_cs_SedGrid.to_csv(Model_cs_SedGrid_xyz, header=False, sep= " ", index=False)
    else:
        Model_cs_SedGrid_xyz=cs_Dir+"/Modeled_cs_SedGrid_%dMa_%dkm.xyz" %(age, stencilWidth)
        model_cs_SedGrid.to_csv(Model_cs_SedGrid_xyz, header=False, sep= " ", index=False)
        
    model_non_cs_SedGrid = non_cs_SedGrid_df[np.isfinite(non_cs_SedGrid_df['Thickness_km'])]
    model_non_cs_SedGrid['exponNumCol'] = np.random.exponential(scale=non_cs_scale, size=model_non_cs_SedGrid.shape[0])
    model_non_cs_SedGrid.drop(['Thickness_km'], axis=1, inplace=True)
    if Clip == 1:
        Model_non_cs_SedGrid_xyz=non_cs_Dir+"/Modeled_non_cs_SedGrid_%dMa_%dkm_Clipped%dkm.xyz" %(age, stencilWidth, clip_lim)
        model_non_cs_SedGrid['exponNumCol'] = np.where(model_non_cs_SedGrid['exponNumCol'] >= clip_lim, clip_lim, model_non_cs_SedGrid['exponNumCol'])
        model_non_cs_SedGrid.to_csv(Model_non_cs_SedGrid_xyz, header=False, sep= " ", index=False)
    else:
        Model_non_cs_SedGrid_xyz=non_cs_Dir+"/Modeled_non_cs_SedGrid_%dMa_%dkm.xyz" %(age, stencilWidth)
        model_non_cs_SedGrid.to_csv(Model_non_cs_SedGrid_xyz, header=False, sep= " ", index=False)
    
    # -- Combine three models into one
    TotalThickness_SedGrid_df = model_cs_SedGrid.append([model_non_cs_SedGrid])
    TotalThickness_SedGrid_df.to_csv(Model_TotalThickness_SedGrid_xyz, header=False, sep= " ", index=False)

    # Add model info to list
    TotalThickness_list.append(TotalThickness_SedGrid_df['exponNumCol'].sum())
    mean_list.append(TotalThickness_SedGrid_df['exponNumCol'].mean())
    
      # -- Convert modeled total sediment thickness xyz to grd
    cmd="xyz2grd -V %s -Rg -I1 -G%s" %(Model_TotalThickness_SedGrid_xyz, Model_TotalThickness_SedGrid)
    print cmd
    os.system(cmd)

SedModeldf = pd.DataFrame({'Age':reconstruction_times, 'mean_thicknes':mean_list})
if Clip == 1:
    SedModeldf.to_csv(workDir+"/SedModelFinal_%s-%sMa_%dkm_Clipped%dkm.csv" %(mintime, maxtime, stencilWidth, clip_lim))
else:
    SedModeldf.to_csv(workDir+"/SedModelFinal_%s-%sMa_%dkm.csv" %(mintime, maxtime, stencilWidth))

